In [1]:
#Necessary libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

In [2]:
##Get the dataset
from sklearn.datasets import load_boston
bos = load_boston()

In [3]:
##Initialize dataset
bos_init = pd.DataFrame(bos.data)

In [4]:
###Preview of the top five rows
bos_init.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [5]:
##Import feature names from the dataset
bos_init.colums = bos.feature_names
bos_init.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [6]:
##Further need to predict price, create a target variable for the ML model
bos_init['Price'] = bos.target

In [7]:
##The ML model will need a numerical value rather than categorical value for the predection of price; check if there are any NULL values
bos_init.isnull().sum()

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
Price    0
dtype: int64

In [8]:
##Check if the values are categorical
bos_init.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       506 non-null    float64
 1   1       506 non-null    float64
 2   2       506 non-null    float64
 3   3       506 non-null    float64
 4   4       506 non-null    float64
 5   5       506 non-null    float64
 6   6       506 non-null    float64
 7   7       506 non-null    float64
 8   8       506 non-null    float64
 9   9       506 non-null    float64
 10  10      506 non-null    float64
 11  11      506 non-null    float64
 12  12      506 non-null    float64
 13  Price   506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


In [ ]:
#Creating the ML model for the Price prediction
##Step 1: Separate features and target variables
##Step 2: Split the dataset into training and testing dataset
##Continue creating the ML model

In [10]:
##Separate features and target variables
x = bos_init.drop(['Price'], axis = 1)
y = bos_init['Price'] 

In [15]:
##Split the dataset into training and testing dataset
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 1)
print('Shape of x_train \n', x_train.shape, '\n\n Shape of x_test \n', x_test.shape, '\n-------------\n')
print('Shape of y_train \n', y_train.shape, '\n\n Shape of y_test \n', y_test.shape)

Shape of x_train 
 (404, 13) 

 Shape of x_test 
 (102, 13) 
-------------

Shape of y_train 
 (404,) 

 Shape of y_test 
 (102,)


In [16]:
##Creating the ML model
from sklearn.ensemble import RandomForestRegressor
classifier = RandomForestRegressor()
classifier.fit(x_train, y_train)

RandomForestRegressor()

Model evaluation; evaluate the model performance for training and test dataset

In [19]:
##Training dataset model evaluation
print('Training dataset model evaluation results \n')
predict_train = classifier.predict(x_train)
print('R^2: ', metrics.r2_score(y_train, predict_train))
print('Mean Abs Error: ', metrics.mean_absolute_error(y_train, predict_train))
print('Mean Squared Error: ', metrics.mean_squared_error(y_train, predict_train))
print('Root Mean Square Error: ', np.sqrt(metrics.mean_squared_error(y_train, predict_train)), '\n')

##Test dataset model evaluation
print('\nTraining dataset model evaluation results \n')
predict_test = classifier.predict(x_test)
print('R^2: ', metrics.r2_score(y_test, predict_test))
print('Mean Abs Error: ', metrics.mean_absolute_error(y_test, predict_test))
print('Mean Squared Error: ', metrics.mean_squared_error(y_test, predict_test))
print('Root Mean Square Error: ', np.sqrt(metrics.mean_squared_error(y_test, predict_test)))

Training dataset model evaluation results 

R^2:  0.9832366322595792
Mean Abs Error:  0.8056212871287117
Mean Squared Error:  1.3541680618811858
Root Mean Square Error:  1.1636872697942457 


Training dataset model evaluation results 

R^2:  0.9197906061986997
Mean Abs Error:  2.2218823529411758
Mean Squared Error:  7.926885000000001
Root Mean Square Error:  2.815472429273638


Save and use the ML model.
Serialization and Deserialization mechanism will be useful to store the ML object model in byte system and the other way round

In [25]:
#Save the model to a file
import pickle
with open('model/model.pkl', 'wb') as file:
    pickle.dump(classifier, file)

#Save the Columns
model_columns = list(x.columns)
with open('model/model_columns.pkl', 'wb') as file:
    pickle.dump(model_columns, file)

API Creation for the ML model

Step 1: pip install Flask

In [27]:
from flask import Flask
app = Flask(__name__)
@app.route('/', methods=['GET, 'POST'])

def main():
    return 'Price Prediction for Boston House'